In [5]:
import pandas as pd
from prompts import prompts
from tqdm import tqdm


# 엑셀 파일 불러오기
df_essay = pd.read_excel('/home/jinhee/NC/AES/datasets/asap-aes/training_set_rel3.xlsx')  # 예: 'data.xlsx'
df_essay_info = pd.read_excel('/home/jinhee/NC/AES/datasets/asap-aes/Essay_Set_Descriptions/Essay_Set_Descriptions/essay_set_descriptions.xlsx')
df_trait_promt1 = pd.read_csv('/home/jinhee/NC/AES/datasets/asap++/Prompt-1.csv')
df_trait_promt2 = pd.read_csv('/home/jinhee/NC/AES/datasets/asap++/Prompt-2.csv')
df_trait_promt3 = pd.read_csv('/home/jinhee/NC/AES/datasets/asap++/Prompt-3.csv')
df_trait_promt4 = pd.read_csv('/home/jinhee/NC/AES/datasets/asap++/Prompt-4.csv')
df_trait_promt5 = pd.read_csv('/home/jinhee/NC/AES/datasets/asap++/Prompt-5.csv')
df_trait_promt6 = pd.read_csv('/home/jinhee/NC/AES/datasets/asap++/Prompt-6.csv')


# 각 행을 dict로 변환하고 리스트에 담기
df_essay_list = df_essay.to_dict(orient='records')
df_essay_info_list = df_essay_info.to_dict(orient='records')
df_trait_promt1_list = df_trait_promt1.to_dict(orient='records')
df_trait_promt2_list = df_trait_promt2.to_dict(orient='records')
df_trait_promt3_list = df_trait_promt3.to_dict(orient='records')
df_trait_promt4_list = df_trait_promt4.to_dict(orient='records')
df_trait_promt5_list = df_trait_promt5.to_dict(orient='records')
df_trait_promt6_list = df_trait_promt6.to_dict(orient='records')


preprocessed_essays = []

# 결과 출력 (선택)
for item in tqdm(df_essay_list):
    if item['essay_set'] == 1:
        score = {'holistic_score':item['domain1_score']/2}
        traits = next(filter(lambda i: i["Essay ID"] == item['essay_id'], df_trait_promt1_list), None)
        if traits is not None:
            _traits = {k: v for k, v in traits.items() if k != "Essay ID"}
            score['trait_score'] = _traits
        else:
            score['trait_score'] = None

    elif item['essay_set'] == 2:
        score = {'holistic_score':item['domain1_score']+item['domain2_score']}
        traits = next(filter(lambda i: i["Essay ID"] == item['essay_id'], df_trait_promt2_list), None)
        if traits is not None:
            _traits = {k: v for k, v in traits.items() if k != "Essay ID"}
            score['trait_score'] = _traits
        else:
            score['trait_score'] = None

    elif item['essay_set'] == 3:
        score = {'holistic_score':item['domain1_score']}
        traits = next(filter(lambda i: i["Essay ID"] == item['essay_id'], df_trait_promt3_list), None)
        if traits is not None:
            _traits = {k: v for k, v in traits.items() if k != "Essay ID"}
            score['trait_score'] = _traits
        else:
            score['trait_score'] = None

    elif item['essay_set'] == 4:
        score = {'holistic_score':item['domain1_score']}
        traits = next(filter(lambda i: i["Essay ID"] == item['essay_id'], df_trait_promt4_list), None)
        if traits is not None:
            _traits = {k: v for k, v in traits.items() if k != "Essay ID"}
            score['trait_score'] = _traits
        else:
            score['trait_score'] = None

    elif item['essay_set'] == 5:
        score = {'holistic_score':item['domain1_score']}
        traits = next(filter(lambda i: i["Essay ID"] == item['essay_id'], df_trait_promt5_list), None)
        if traits is not None:
            _traits = {k: v for k, v in traits.items() if k != "Essay ID"}
            score['trait_score'] = _traits
        else:
            score['trait_score'] = None

    elif item['essay_set'] == 6:
        score = {'holistic_score':item['domain1_score']}
        traits = next(filter(lambda i: i["Essay ID"] == item['essay_id'], df_trait_promt6_list), None)
        if traits is not None:
            _traits = {k: v for k, v in traits.items() if k != "Essay ID"}
            score['trait_score'] = _traits
        else:
            score['trait_score'] = None

    elif item['essay_set'] == 7:
        score = {'holistic_score':item['domain1_score']/2, 
                 'trait_score':{'Ideas':(item['rater1_trait1']+item['rater2_trait1'])/2, 
                                'Organization':(item['rater1_trait2']+item['rater2_trait2'])/2, 
                                'Style':(item['rater1_trait3']+item['rater2_trait3'])/2, 
                                'Conventions':(item['rater1_trait4']+item['rater2_trait4'])/2}}
    elif item['essay_set'] == 8: # holistic_score(~30)와 trait_score(각 ~6)의 기준이 좀 달라서 범위도 달라짐 
        score = {'holistic_score':(item['rater1_domain1']+item['rater2_domain1'])/2,
         'trait_score':{'Ideas and Content':(item['rater1_trait1']+item['rater2_trait1'])/2, 
                        'Organization':(item['rater1_trait2']+item['rater2_trait2'])/2, 
                        'Voice':(item['rater1_trait3']+item['rater2_trait3'])/2, 
                        'Word Choice':(item['rater1_trait4']+item['rater2_trait4'])/2, 
                        'Sentence Fluency':(item['rater1_trait5']+item['rater2_trait5'])/2, 
                        'Conventions':(item['rater1_trait6']+item['rater2_trait6'])/2}}
    
    info = next(filter(lambda i: i["essay_set"] == item['essay_set'], df_essay_info_list))
    type_of_essay = info['type_of_essay']
    grade_level = info['grade_level']

    prompt = next(filter(lambda p: p["essay_set"] == item['essay_set'], prompts))
    source_text = prompt['source_text']
    prompt_text = prompt['prompt_text']
    rubric_guidelines = prompt['rubric_guidelines']
    trait_guidlines = prompt['trait_guidelines']

    essay_id = item['essay_id']
    essay_set = item['essay_set']
    essay = item['essay']
 
    preprocessed_essays.append({'essay_id':essay_id,
                                'essay_set':essay_set,
                                'type_of_essay':type_of_essay,
                                'grade_level':grade_level,
                                'source_text':source_text,
                                'prompt_text':prompt_text,
                                'essay':essay,
                                'rubric_guidelines':rubric_guidelines,
                                'trait_guidlines':trait_guidlines,
                                'score':score
                                })


100%|██████████| 12978/12978 [00:01<00:00, 7465.02it/s]


In [17]:
preprocessed_essays[4724]

{'essay_id': 7122,
 'essay_set': 3,
 'type_of_essay': 'source dependent responses',
 'grade_level': 10,
 'source_text': 'ROUGH ROAD AHEAD: Do Not Exceed Posted Speed Limit\nby Joe Kurmaskie\nFORGET THAT OLD SAYING ABOUT NEVER taking candy from strangers. No, a better piece of advice for the solo cyclist would be, “Never accept travel advice from a collection of old-timers who haven’t left the confines of their porches since Carter was in office.” It’s not that a group of old guys doesn’t know the terrain. With age comes wisdom and all that, but the world is a fluid place. Things change. \nAt a reservoir campground outside of Lodi, California, I enjoyed the serenity of an early-summer evening and some lively conversation with these old codgers. What I shouldn’t have done was let them have a peek at my map. Like a foolish youth, the next morning I followed their advice and launched out at first light along a “shortcut” that was to slice away hours from my ride to Yosemite National Park.\

In [ ]:
import json
with open('preprocessed_essays.jsonl', 'w', encoding='utf-8') as f:
    for item in preprocessed_essays:
        json_line = json.dumps(item, ensure_ascii=False)
        f.write(json_line + '\n')